<a href="https://colab.research.google.com/github/YuanSnowing/colab/blob/main/LucidDreamer_Gaussian_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone --recursive -b dev https://github.com/camenduru/LucidDreamer-Gaussian
!pip install peft diffusers scipy numpy imageio[ffmpeg] opencv-python Pillow open3d torch==2.0.1  torchvision==0.15.2 gradio omegaconf
!pip install timm==0.6.7
!pip install plyfile==0.8.1
!pip install -q diffusers accelerate gradio open3d plyfile timm==0.6.12
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl

!apt install libglm-dev
!pip install /content/LucidDreamer-Gaussian/submodules/depth-diff-gaussian-rasterization-min
!pip install /content/LucidDreamer-Gaussian/submodules/simple-knn

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/ZoeDepth/resolve/main/ZoeD_M12_N.pt -d /root/.cache/torch/hub/checkpoints -o ZoeD_M12_N.pt

!sed -i 's/\(new_height, new_width\)/int(new_height), int(new_width)/g' /content/LucidDreamer-Gaussian/ZoeDepth/zoedepth/models/base_models/midas.py


/content
Cloning into 'LucidDreamer-Gaussian'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 442 (delta 124), reused 103 (delta 103), pack-reused 297 (from 1)
Receiving objects: 100% (442/442), 62.50 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (187/187), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:

FileNotFoundError: [Errno 2] No such file or directory: '/content/./ZoeDepth/hubconf.py'

In [ ]:
import torch
torch.hub.load('LucidDreamer-Gaussian/ZoeDepth', 'ZoeD_N', source='local', pretrained=True).to('cuda')
!sed -i 's/\(new_height, new_width\)/int(new_height), int(new_width)/g' /root/.cache/torch/hub/intel-isl_MiDaS_master/midas/backbones/beit.py

%cd /content/LucidDreamer-Gaussian
!python app.py

img_size [384, 512]


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _V